In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from keras.layers import Dropout
from keras.engine.sequential import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
data_path = 'data/2017-20211231_15min.csv'
data = pd.read_csv(data_path)
data = data[(data['timestamp'] >= '2021-01-01') & (data['timestamp'] <= '2021-12-31')]
data = data.drop(['timestamp'], axis=1)

data.head()

,price_open,price_high,price_low,price_close,volume_traded,MACD,SMA,OBV,RSI,MFI,vader_pos,vader_neg,vader_neu,vader_compound,pos_count,neg_count,neu_count
116928,28999.63,29086.90,28774.64,28834.74,71.484592,1922.975194,28985.870,-39285.54260,50.545732,17.900402,0.0,0.0,1.0,0.0,0.0,0.0,0.0
116929,28812.07,28947.70,28800.92,28910.87,74.896062,1929.048243,28941.158,-39210.64654,50.190206,34.905701,0.0,0.0,1.0,0.0,0.0,0.0,0.0
116930,28900.11,29017.21,28900.11,29017.21,154.640031,1937.531206,28932.038,-39056.00651,50.433165,8.534226,0.0,0.0,1.0,0.0,0.0,0.0,0.0
116931,29005.98,29099.00,28971.91,29056.94,96.944090,1940.700550,28962.510,-38959.06242,49.376523,8.774641,0.0,0.0,1.0,0.0,0.0,0.0,0.0
116932,29070.66,29437.83,29010.36,29413.41,149.063026,1969.136904,29046.634,-38809.99939,54.127520,53.275767,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [3]:
data_features = data[['price_close', 'vader_pos', 'vader_neg']]

data_labels = data['price_close']

scaler_feature = MinMaxScaler(feature_range=(0, 1))
scaler_label = MinMaxScaler(feature_range=(0, 1))

feature = scaler_feature.fit_transform(data_features.to_numpy())
label = scaler_label.fit_transform(data_labels.to_numpy().reshape(-1, 1))

In [4]:
def split_data(f, l, delay):
    feature, label = [], []

    for i in range(len(f) - delay):
        feature.append(f[i: i + delay])
        label.append(l[i + delay-1])
        
    return np.array(feature), np.array(label).squeeze()

window_size = 1

feature, label = split_data(feature, label, window_size)


X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)

(27954, 1, 3) (6989, 1, 3)


In [5]:
model = \
    keras.Sequential([
        layers.Bidirectional(layers.GRU(50)),
        layers.Dropout(0.2),
        layers.Dense(1, activation='tanh'),
    ])

2022-11-04 19:23:31.273698: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:23:31.326050: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:23:31.326445: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 19:23:31.327899: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [6]:
EPOCHS = 500
BATCH_SIZE = 8

model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=3e-6), metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(X_train, y_train, 
                batch_size=BATCH_SIZE, 
                epochs=EPOCHS, 
                shuffle=False,
                )

2022-11-04 19:23:32.207393: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/500


2022-11-04 19:23:34.911037: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8400


3495/3495 [==============================] - 9s 2ms/step - loss: 0.2399 - root_mean_squared_error: 0.4898
Epoch 2/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.1286 - root_mean_squared_error: 0.3586
Epoch 3/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0652 - root_mean_squared_error: 0.2553
Epoch 4/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0389 - root_mean_squared_error: 0.1973
Epoch 5/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0305 - root_mean_squared_error: 0.1748
Epoch 6/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0253 - root_mean_squared_error: 0.1592
Epoch 7/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0205 - root_mean_squared_error: 0.1431
Epoch 8/500
3495/3495 [==============================] - 6s 2ms/step - loss: 0.0160 - root_mean_squared_error: 0.1266
Epoch 9/500
3495/3495 [==============================] - 6s 2ms/step

In [7]:
prediction = np.array(model.predict(X_test))
pred_prices = scaler_label.inverse_transform(prediction)
true_prices = scaler_label.inverse_transform(y_test.reshape(-1,1))

mse = mean_squared_error(pred_prices, true_prices)
rmse = mean_squared_error(pred_prices, true_prices, squared=False)
mae = mean_absolute_error(pred_prices, true_prices)
mape = mean_absolute_percentage_error(pred_prices, true_prices)

print('mse:\t', mse)
print('rmse:\t', rmse)
print('mae:\t', mae)
print('mape:\t', mape)

mse:	 276954.85286691337
rmse:	 526.26500251006
mae:	 368.2610092086672
mape:	 0.00763666265029591
